In [108]:
!pip install scikit-surprise

In [109]:
import surprise

print(surprise.__version__)

1.1.1


## 1. 내장된 무비렌즈 데이터로 surprise 패키지 사용해보기

In [110]:
from surprise import SVD      # MF(행렬 분해) 알고리즘
from surprise import Dataset  # 내장 데이터 불러오는 라이브러리
from surprise import accuracy # accuracy의 rmse메소드로 예측 에러 평가
from surprise.model_selection import train_test_split  # 학습셋, 테스트셋 분리

#### 내장 데이터 로드하고 데이터를 학습셋, 테스트셋으로 분리

In [111]:
# 영화 데이터 10만개가 들어있는 "ml-100k" 데이터 로드하기
data = Dataset.load_builtin('ml-100k') 

# 로드된 데이터를 학습셋, 테스트셋(25%)으로 나누기
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

In [112]:
trainset

#### 행렬 분해 알고리즘으로 SVD 객체를 생성하고 학습 수행

In [113]:
algo = SVD()
algo.fit(trainset) 

#### 테스트 데이터에 예상 평점 데이터 예측하기

In [114]:
# 테스트 데이터 사용자 아이템에 대한 예상 평점 리스트 구하기
predictions = algo.test( testset )

print('prediction type :',type(predictions), ' size:',len(predictions))
print('25000개 예상 평점 리스트의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
25000개 예상 평점 리스트의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.573627661907186, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.982288132489444, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.129163604649168, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.5201769450056286, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.697214472015401, details={'was_impossible': False})]

In [115]:
# 간단하게 확인해보기
[(pred.uid, pred.iid, pred.r_ui, pred.est) for pred in predictions[:3] ]

[('120', '282', 4.0, 3.573627661907186),
 ('882', '291', 4.0, 3.982288132489444),
 ('535', '507', 5.0, 4.129163604649168)]

In [116]:
# 예측 평점들의 에러인 rmse를 구해보면,
accuracy.rmse(predictions)

RMSE: 0.9488


0.9488016553018004

#### predict 메소드는 개별 사용자, 아이템에 대한 예측 평점 계산

In [117]:
# 주의 : user_id, item_id는 문자열 type으로 입력해야 함.
uid = str(196)
iid = str(302)

pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.33   {'was_impossible': False}


## 2. 실제 영화 평점 데이터로 surprise 패키지 사용해보기

In [118]:
from surprise import SVD      # 행렬 분해 알고리즘 
from surprise import Dataset  # 내장 데이터 불러오는 라이브러리
from surprise import accuracy # rmse로 예측 에러 평가
from surprise.model_selection import train_test_split  # 학습셋, 테스트셋 분리

#### Surprise 주요 모듈 소개

In [119]:
import pandas as pd

# 원본 csv 파일은 hearder가 있는데, surprise를 적용하려면 header를 없애줘야 한다.
ratings = pd.read_csv('./ml-latest-small/ratings.csv')

In [120]:
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [121]:
%%time

# ratings_noh.csv 파일로 저장하면 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)

CPU times: user 245 ms, sys: 12.4 ms, total: 257 ms
Wall time: 259 ms


## 데이터 로딩하기(load_from_file이용, reader로 파일 포멧 지정)

In [122]:
from surprise import Reader
# reader로 파일 포멧 지정하기( 컬럼 명 4개 지정, 콤마로 구분, 평점 범위는 0.5 ~ 5점 )
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

# 데이터 로딩하기
data = Dataset.load_from_file('./ml-latest-small/ratings_noh.csv', reader=reader)

In [123]:
# 데이터를 학습셋, 테스트셋으로 나누기
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [124]:
# SVD 알고리즘 적용(잠재 요인의 차원 수를 50개, 수행시마다 동일한 결과 도출을 위해 random_state 설정)
algo = SVD(n_factors=50, random_state=0)

In [125]:
# 학습셋으로 학습
algo.fit(trainset)

In [88]:
%%time
# 테스트셋으로 예상 평점 예측
predictions = algo.test(testset)

CPU times: user 151 ms, sys: 5.41 ms, total: 157 ms
Wall time: 154 ms


In [126]:
predictions[:5]

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.573627661907186, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.982288132489444, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.129163604649168, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.5201769450056286, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.697214472015401, details={'was_impossible': False})]

In [127]:
# RMSE 평가
accuracy.rmse(predictions)

RMSE: 0.9488


0.9488016553018004

## 3. Cross Validation(교차 검증)과 GridSearchCV(하이퍼 파라미터 튜닝)

In [128]:
%%time
from surprise.model_selection import cross_validate # 교차 검증 : 과적합 피한 모델 학습이 가능하다.

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('./ml-latest-small/ratings.csv') # reading data in pandas df

# 데이터 샘플링 (2천만 건 데이터를 20만 건으로)
ratings = ratings.sample(frac=0.01)

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# SVD 알고리즘 
algo = SVD(random_state=0) 

# 교차 검증
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1635  0.9118  0.9664  1.0164  0.9142  0.9945  0.0929  
MAE (testset)     0.9383  0.6935  0.7780  0.8193  0.7218  0.7902  0.0860  
Fit time          0.05    0.05    0.05    0.05    0.05    0.05    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    
CPU times: user 294 ms, sys: 20.6 ms, total: 314 ms
Wall time: 320 ms


{'test_rmse': array([1.16354148, 0.91176456, 0.96643903, 1.0164312 , 0.91424376]),
 'test_mae': array([0.93829057, 0.69349867, 0.77804873, 0.81931771, 0.72177862]),
 'fit_time': (0.05171394348144531,
  0.04768824577331543,
  0.04636383056640625,
  0.04507803916931152,
  0.04643678665161133),
 'test_time': (0.0010311603546142578,
  0.0010640621185302734,
  0.0009279251098632812,
  0.000988006591796875,
  0.0009989738464355469)}

### 그리드 서치 CV 이용

In [129]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# GridSearchCV 세팅 : CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs

In [130]:
%%time
# GridSearchCV로 학습
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# 20만 건 데이터 기준 실행시간 (원본 데이터 2천만 건이면 8분~10분 정도 걸림)

0.9906711065791266
{'n_epochs': 60, 'n_factors': 50}
CPU times: user 2.38 s, sys: 21.8 ms, total: 2.4 s
Wall time: 2.42 s


## 4. Surprise 를 이용한 개인화 영화 추천 시스템 구축

In [94]:
# # 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
# data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# algo = SVD(n_factors=50, random_state=0)
# algo.fit(data)

### 전체 데이터로 학습 진행(학습셋, 테스트셋 나누지 않고)

In [131]:
from surprise.dataset import DatasetAutoFolds  # 데이터 세트 전체를 학습 데이터로 사용할 수 있게 해주는 라이브러리

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습데이터로 생성함.
trainset = data_folds.build_full_trainset()

In [132]:
# SVD 협업필터링으로 추천모델 학습(하이퍼 파라미터는 앞서 그리드서치로 구한 것들)
algo = SVD(n_epochs=40, n_factors=50, random_state=0)

algo.fit(trainset)

In [133]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [134]:
movieIds = ratings[ratings['userId']==9]['movieId']
movieIds

1095    4131
Name: movieId, dtype: int64

In [153]:
# userId=9 의 movieId 확인. 
movieIds = ratings[ratings['userId']==9]
movieIds

,userId,movieId,rating,timestamp
1095,9,4131,1.0,1044657221


In [154]:
print(movies[movies['movieId']==42])

    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [160]:
# predict 메소드를 사용해서 예측 평점 구하기
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.11   {'was_impossible': False}


### 특정 사용자가 아직 안 본 전체 영화 추출 후 예측 평점 순으로 영화 추천

In [161]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 1 추천대상 영화수: 9741 전체 영화수: 9742


In [162]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화(27243개)에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [163]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print("")
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 1 추천대상 영화수: 9741 전체 영화수: 9742

##### Top-10 추천 영화 리스트 #####
Seven (a.k.a. Se7en) (1995) : 4.92978369537401
Usual Suspects, The (1995) : 4.9193016642129885
Star Wars: Episode IV - A New Hope (1977) : 4.796454763203678
Pulp Fiction (1994) : 4.776135585777866
Silence of the Lambs, The (1991) : 4.752482814555706
Godfather, The (1972) : 4.6709662461230135
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.670421590751661
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981) : 4.654583188673701
Lord of the Rings: The Fellowship of the Ring, The (2001) : 4.570164451925616
Lord of the Rings: The Return of the King, The (2003) : 4.5456751872004455
